# Practical 7 - Part 2A
This project explores the geometry of a single camera. The aim is to take several points on
a plane, and predict where they will appear in the camera image. Based on these observed
points, we will then try to re-estimate the Euclidean transformation relating the plane and
the camera. In practical 2B we will use this code to draw a wireframe cube
on an augmented reality marker.   You should use this
template for your code and fill in the missing sections marked "TO DO"

# Import libraries 

In [148]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from numpy.linalg import inv
from numpy.linalg import det

# TO DO - fill in projectiveCamera and estimatePlanePose functions  (you will have to utilise your solutions for solveAXEqualsZero and calcBestHomography from Part 1)

In [167]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    u,s,v = np.linalg.svd(A)
    v_last = v[-1,:]
    return v_last

In [168]:
def solve_rotation(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    u,s,v = np.linalg.svd(A)
    matrix = np.array([[1,0],[0,1],[0,0]])
    result = np.matmul(np.matmul(u,matrix),v)
    return result

In [169]:
# This function should apply the direct linear transform (DLT) algorithm to calculate the best 
# homography that maps the points in pts1Cart to their corresonding matching in pts2Cart
def calcBestHomography(XCart, XCamCart):    


    # TO DO: 
    # First convert points into homogeneous representation
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    XCamHom = np.concatenate((XCamCart, np.ones((1,XCamCart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (n_points,9)
    num_examples = 10
    A = np.zeros((num_examples,9))
    
    #Populate the matrix
    for i in range(5):
        uv = XHom[:, i]
        xy = XCamHom[:, i]
        A[2 * i,     :] = np.hstack([[0, 0, 0], -uv.T, uv.T * xy[1]])
        A[2 * i + 1, :] = np.hstack([uv.T, [0, 0, 0], -uv.T * xy[0]])
    
    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    #print(h.shape)
    
    # Reshape h into the matrix H, values of h go first into rows of H
    H = np.reshape(h, (3, 3))
    
    return H

In [170]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    # First convert points into homogeneous representation
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    XCamHom = np.matmul(T,XHom)
    
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    XCamHom_norm = XCamHom[:-1,:]
    
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    xImHom = np.matmul(K,XCamHom_norm)
    
    # TO DO: Convert points back to Cartesian coordinates xImCart
    XImCart = xImHom[0:2,:] / np.tile([xImHom[2,:]],(2,1))
    
    return XImCart


In [171]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.matmul(inv(K),XImHom)
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    
    XCamCart = XCamHom[0:2,:] / np.tile([XCamHom[2,:]],(2,1))
    
    XCart = XCart[:-1,:]
    H = calcBestHomography(XCart,XCamCart)
    
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD       
    Rest = solve_rotation(H[:,:2])

    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    Rest_2 = np.cross(Rest[:,0],Rest[:,1])
    Rest_2 = Rest_2.reshape((3,1))
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    Rest_final = np.hstack((Rest,Rest_2))
    
    if det(Rest_final)<0:
        Rest_final[:,-1] = Rest_final[:,-1] * -1
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    scaling_factor = np.sum(H[:,:2]/Rest_final[:,:2])/6
    t = H[:,-1]/scaling_factor 
    t = t.reshape((3,1))
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[-1,:] < 0:
        t[:,:] = t[:,:] * -1
        Rest_final[:,:2] = Rest_final[:,:2] * -1
            
    # TO DO: Assemble transformation into matrix form
    T = np.hstack((Rest_final,t))
    
    return T 

# Once you have completed these functions, use them to estimate the transformation from the plane co-ordinate system to the camera co-ordinate system (i.e. the extrinsic matrix)

In [177]:
# We assume that the intrinsic camera matrix K is known and has values
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are four known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
  
# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera" - body of function appears below:

XImCart = projectiveCamera(K,T,XCart)

# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCart = XImCart + np.random.normal(1,size=(2,5))

# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function "estimate plane pose"
TEst = estimatePlanePose(XImCart,XCart,K)

# If you have got this correct, Test should closely resemble T above

(3, 3)


In [178]:
TEst

array([[  9.85520172e-01,  -4.93647932e-02,   1.62213157e-01,
          4.60523019e+01],
       [ -1.62275327e-01,  -5.51968840e-01,   8.17922440e-01,
          7.00795899e+01],
       [  4.91600358e-02,  -8.32402256e-01,  -5.51987114e-01,
          5.01459511e+02]])

In [176]:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])